# ETL Practice Project
**Project Scenario** 
<br>
An international firm that is looking to expand its business in different countries across the world has recruited you. You have been hired as a junior Data Engineer and are tasked with:

1. Creating an automated script that can extract the list of all countries:
- In order of their GDPs in billion USDs
- Rounded to 2 decimal places
- As logged by the International Monetary Fund (IMF). Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated. The required data seems to be available on the URL mentioned here: <a href=https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29>List of countries by GDP (nominal)</a>

2. The required information needs to be made accessible as:
- A CSV file Countries_by_GDP.csv
- A table Countries_by_GDP in a database file World_Economies.db with attributes Country and GDP_USD_billion

3. Your boss wants you to demonstrate the success of this code by:
- Running a query on the database table to display only the entries with more than a 100 billion USD economy
- Log in a file with the entire process of execution named etl_project_log.txt

**You must create a Python code 'etl_project_gdp.py' that performs all the required tasks**


**Project Objectives**
<br>
You have to complete the following tasks for this project:
1. Write a data extraction function to retrieve the relevant information from the required URL.
1. Transform the available GDP information into 'Billion USD' from 'Million USD'
1. Load the transformed information to the required CSV file and as a database file
1. Run the required query on the database
1. Log the progress of the code with appropriate timestamps.

<hr>

**Setup**

In [1]:
# import required libraries
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np


In [2]:
#define empty entities
db_name = 'world_economies.db'
table_name = 'countries_by_gdp'
csv_path = './countries_by_gdp.csv'

In [3]:
# define source and source structure
url = 'https://web.archive.org/web/20230902185326/https:/en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ['Country', 'GDP (Millions, USD)']
column_indices = [0, 2]  # target columns from which data will be extracted


**Testing Extract**

In [ ]:
# parse HTML from url
html_content = requests.get(url).content
data = BeautifulSoup(html_content, 'html.parser') # choose content to also bring in tags
table = data.find_all('table')[2] # get content related to table at index 2
rows = table.find_all('tr') # get all rows
rows = rows[2:] # remove first two header rows
rows

In [8]:
# filter cells from the specified columns into a list
filtered_cells = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) == 0: #remember to check number of cells in a row - many web tables have invisible merged rows
        continue
    if row.find('a') is None:
        continue
    selected_cells = [cells[i] for i in column_indices if i < len(cells)]
    filtered_cells.append(selected_cells)

In [131]:
# extract data from filtered cells into a pandas dataframe
df = pd.DataFrame(columns=table_attribs)
for row in filtered_cells:
    data_dict = {table_attribs[0]: row[0].text.strip(), table_attribs[1]: row[1].text.strip()}
    df1 = pd.DataFrame([data_dict], index=[0])
    df = pd.concat([df, df1], ignore_index=True)

In [138]:
# replace special character in GDP column with 0
df[table_attribs[1]] = df[table_attribs[1]].replace('—', 0)

In [25]:
def extract(url,table_attribs,column_indices):
    # parse HTML from url
    html_content = requests.get(url).content
    data = BeautifulSoup(html_content, 'html.parser') # choose content to also bring in tags
    table = data.find_all('table')[2] # get content related to table at index 2
    rows = table.find_all('tr') # get all rows
    rows = rows[2:] # remove first two header rows

    # filter cells from the specified columns into a list
    filtered_cells = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 0: # remember to check number of cells in a row - many web tables have invisible merged rows
            continue
        if row.find('a') is None: # rows not hyperlinked to a country, namely World, are excluded
            continue
        selected_cells = [cells[i] for i in column_indices if i < len(cells)]
        filtered_cells.append(selected_cells)

    # extract data from filtered cells into a pandas dataframe
    df = pd.DataFrame(columns=table_attribs)
    for row in filtered_cells:
        data_dict = {table_attribs[0]: row[0].text.strip(), table_attribs[1]: row[1].text.strip()}
        df1 = pd.DataFrame([data_dict], index=[0])
        df = pd.concat([df, df1], ignore_index=True)
        
    return df

In [ ]:
data = extract(url,table_attribs,column_indices)
data

**Testing Transform**

In [48]:
# clean string data in GDP column
data['GDP (Millions, USD)'] = data['GDP (Millions, USD)'].str.replace(',', '') # remove commas from string
data['GDP (Millions, USD)'] = data['GDP (Millions, USD)'].replace('—', 0) # convert special character to zero


In [ ]:
# convert millions to billions, rounded to two decimal places
data['GDP (Millions, USD)'] = round(data['GDP (Millions, USD)'].astype(float) / 1000,2) # convert string to float to run arithmetic operations
data.rename(columns={'GDP (Millions, USD)': 'GDP (Billions, USD)'}, inplace=True)  # rename column to reflect new values
print(data)


In [51]:
def transform(df):
    # clean string data in GDP column
    df['GDP (Millions, USD)'] = df['GDP (Millions, USD)'].str.replace(',', '') # remove commas from string
    df['GDP (Millions, USD)'] = df['GDP (Millions, USD)'].replace('—', 0) # convert special character to zero

    # convert millions to billions, rounded to two decimal places
    df['GDP (Millions, USD)'] = round(df['GDP (Millions, USD)'].astype(float) / 1000,2) # convert string to float to run arithmetic operations
    df.rename(columns={'GDP (Millions, USD)': 'GDP (Billions, USD)'}, inplace=True)  # rename column to reflect new values
    return df

In [52]:
transformed_data = transform(data)
transformed_data

,Country,"GDP (Billions, USD)"
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
208,Anguilla,0.00
209,Kiribati,0.25
210,Nauru,0.15
211,Montserrat,0.00


In [59]:
from importlib import reload
reload(ru)
import repo_utils as ru

onedrive_root = ru.get_onedrive_root()

ru.function_to_square_numbers(4)

ModuleNotFoundError: No module named 'yaml'

**Testing Load**

**Testing Query**

In [ ]:
# Code for ETL operations on Country-GDP data

def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''

''' Here, you define the required entities and call the relevant 
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''